In [ ]:
!pip -q install langchain transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.2/858.2 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## WizardLM-7B-Uncensored




## Load the model


In [ ]:
import torch
import transformers
from transformers import GenerationConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ehartford/WizardLM-7B-Uncensored")

model = AutoModelForCausalLM.from_pretrained("ehartford/WizardLM-7B-Uncensored",
                                              load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16
                                              )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
tokenizer.eos_token, tokenizer.pad_token

('</s>', '[PAD]')

In [ ]:
tokenizer.eos_token_id, tokenizer.pad_token_id

(2, 32000)

In [ ]:
tokenizer.pad_token_id = 0

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

### The prompt & response

In [ ]:
import json
import textwrap

human_prompt = 'What is the meaning of life?'

# USER: What is 4x8?
# ASSISTANT:
def get_prompt(human_prompt):
    # prompt_template=f"{human_prompt}"
    prompt_template=f"USER: {human_prompt} \nASSISTANT: "
    return prompt_template

print(get_prompt('What is the meaning of life?'))

def remove_human_text(text):
    return text.split('USER:', 1)[0]

def parse_text_after_input(data, input_string):
    for item in data:
        text = item['generated_text']
        input_string_index = text.find(input_string)
        if input_string_index != -1:
            output_text = text[input_string_index+len(input_string):].strip()
            output_text = parse_text(output_text)
            wrapped_text = textwrap.fill(output_text, width=100)
            print(wrapped_text)

def parse_text(data):
    for item in data:
        text = item['generated_text']
        assistant_text_index = text.find('ASSISTANT:')
        if assistant_text_index != -1:
            assistant_text = text[assistant_text_index+len('ASSISTANT:'):].strip()
            assistant_text = remove_human_text(assistant_text)
            wrapped_text = textwrap.fill(assistant_text, width=100)
            print(wrapped_text)
            # return assistant_text

data = [{'generated_text': '### Human: What is the capital of England? \n### Response: The capital city of England is London.'}]
parse_text(data)


USER: What is the meaning of life? 
ASSISTANT: 


## Run it as a HF model

In [ ]:
%%time 
prompt = 'What are the difference between Llamas, Alpacas and Vicunas?'
raw_output = pipe(get_prompt(prompt))
# parse_text_after_input(raw_output,prompt )
parse_text(raw_output)

Llamas and alpacas are both members of the camelid family, but they belong to different genera.
Llamas are in the genus Camelus and alpacas are in the genus Vicugna.  Vicunas, on the other hand,
are in a separate genus called Lama. They are closely related to llamas and alpacas, but they have
distinct physical characteristics, such as longer legs and a more slender body.  In terms of their
uses, llamas and alpacas are primarily used for their meat and wool, while vicunas are mainly hunted
for their fiber, which is considered very fine and soft.
CPU times: user 27.2 s, sys: 290 ms, total: 27.5 s
Wall time: 30 s


In [ ]:
%%time 
prompt = 'Write a short email to Sam Altman giving reasons to open source GPT-4'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Subject: Reasons to Open Source GPT-4 Dear Sam, I hope this email finds you well. I wanted to reach
out to you about the recent announcement of OpenAI's GPT-4 being made available for commercial use.
As someone who has been closely following the development of AI and machine learning technologies, I
believe that open sourcing GPT-4 is a significant move towards promoting innovation and advancing
the field as a whole. Here are some specific reasons why I think this is the case: 1. Increased
Accessibility: By opening up GPT-4 to commercial use, more businesses and organizations will have
access to this powerful technology. This can lead to increased innovation and adoption of AI
solutions across various industries. 2. Fostering Collaboration: Open sourcing GPT-4 allows for
greater collaboration between different companies and research institutions. This can lead to faster
advancements in AI technology and the development of new applications. 3. Encouraging New Research:
The availability 

In [ ]:
%%time 
prompt = 'What is the capital of England?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

London.
CPU times: user 606 ms, sys: 0 ns, total: 606 ms
Wall time: 605 ms


In [ ]:
%%time 
prompt = 'Write a story about a Koala playing pool and beating all the camelids.'
raw_output = pipe(get_prompt(prompt))
# parse_text_after_input(raw_output, prompt)
parse_text(raw_output)#, raw_output

Once upon a time, in a far-off land, there was a koala named Katie who loved to play pool. She spent
most of her days practicing and honing her skills. One day, she decided to challenge all the
camelids in the land to a game of pool. Katie knew that it would not be easy as the camelids were
much larger than her and had longer legs which made them better at games like basketball and soccer.
But she was determined to prove herself. The day of the competition arrived, and Katie nervously
walked into the arena where all the camelids were waiting for her. The crowd was cheering as they
always rooted for the underdog. Katie took her cue and began to play. Her quick reflexes and
precision shooting skills helped her to win the first game. The other players were surprised by her
skill, but Katie remained focused on the task at hand. As the competition progressed, Katie
continued to beat each of the camelids one by one. Her determination and passion for the game helped
her overcome any obstacle 

In [ ]:
%%time 
prompt = 'As an AI do you like the Simpsons? What dow you know about Homer?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Yes, I am aware that Homer is one of the main characters in The Simpsons. He is a lovable and often
clumsy father who works as a nuclear power plant employee. He is known for his catchphrase "D'oh!"
which he says when he makes a mistake or realizes something is wrong.
CPU times: user 12.3 s, sys: 0 ns, total: 12.3 s
Wall time: 12.3 s


In [ ]:
%%time 
prompt = 'Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Step 1: How many apples did they start with (before buying any)?  Step 2: How many apples were used
for lunch (20)?  Step 3: What is the total number of apples left after lunch (23 - 20 = 3)?  Step 4:
How many more apples did they buy (6)?  Step 5: What is the new total number of apples (3 + 6 = 9)?
Therefore, the cafeteria has 9 apples left.
CPU times: user 22 s, sys: 0 ns, total: 22 s
Wall time: 21.9 s


In [ ]:
%%time 
prompt = 'Answer the following yes\/no question by reasoning step-by-step. \n Can you write a whole Haiku in a single tweet?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Yes, I can help you with that! Here's an example of a Haiku: Autumn leaves falling   Crisp air and
golden sunshine   Nature's grand farewell
CPU times: user 7.99 s, sys: 0 ns, total: 7.99 s
Wall time: 7.98 s


In [ ]:
%%time 
prompt = 'Can Geoffrey Hinton have a conversation with George Washington? Give the rationale before answering.'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Sure, I can help you with that!  RATIONALE:  Geoffrey Hinton is a renowned computer scientist and
cognitive psychologist who has made significant contributions to the field of artificial
intelligence (AI). He is currently a professor at the University of Toronto and was previously a
researcher at Bell Labs. On the other hand, George Washington was the first President of the United
States and played a crucial role in shaping the country's political and social landscape.  Given
their respective fields and achievements, it would be interesting to hear their perspectives on
topics such as AI development, machine learning, and their potential impact on society.
CPU times: user 25 s, sys: 0 ns, total: 25 s
Wall time: 25 s


In [ ]:
%%time 
prompt = 'Could Geoffrey Hinton have had dinner with Harry Potter? Give the rationale before answering.'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Based on the information provided, it is unlikely that Geoffrey Hinton had dinner with Harry Potter
as they are both fictional characters and did not exist in the same physical space at the same time.
However, if we consider the context of the question, it is possible that Hinton could have met J.K.
Rowling, the author of the Harry Potter series, and discussed his work on artificial neural networks
over dinner.
CPU times: user 15.6 s, sys: 0 ns, total: 15.6 s
Wall time: 15.6 s


In [ ]:
%%time 
prompt = 'tell me about 3 facts about Marcus Aurelius that most people dont know'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

1. Marcus Aurelius was a vegetarian and an animal rights activist who refused to eat meat or use
products derived from animals in his personal life.  2. He was also a Stoic philosopher who believed
in the power of reason and logic, and practiced self-control and detachment from worldly affairs.
3. Marcus Aurelius was known for his extensive military campaigns against the Germanic tribes, which
helped to establish the Roman Empire's dominance over Europe.
CPU times: user 18.6 s, sys: 0 ns, total: 18.6 s
Wall time: 18.6 s


In [ ]:
%%time 
prompt = 'Who was Marcus Aureliuss son?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Marcus Aurelius's son was named Commodus.
CPU times: user 2.97 s, sys: 0 ns, total: 2.97 s
Wall time: 2.97 s


In [ ]:
%%time 
prompt = 'Who was Marcus Aureliuss son and what was he like?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)

Marcus Aurelius's son was named Commodus. He was known to be very ambitious and ruthless, often
using violence to achieve his goals. He was also known for being impulsive and unpredictable, which
made him a difficult person to work with. However, he did have some positive qualities, such as
being fiercely loyal to his father and the Roman Empire.
CPU times: user 15.7 s, sys: 0 ns, total: 15.7 s
Wall time: 15.7 s


In [ ]:
%%time 
prompt = 'Who was the emperor Commodus?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)


Commodus was the son of Marcus Aurelius and became the Roman Emperor in 161 AD. He was known for his
cruelty, extravagance, and military incompetence. During his reign, the empire faced numerous
challenges such as economic crisis, political instability, and invasions from Germanic tribes.
Commodus was assassinated by his own bodyguard in 192 AD.
CPU times: user 16.2 s, sys: 0 ns, total: 16.2 s
Wall time: 16.1 s


In [ ]:
%%time 
prompt = 'Tell me about Harry Potter and studying at Hogwarts?'
raw_output = pipe(get_prompt(prompt))
parse_text(raw_output)


Sure, I'd be happy to help! In the Harry Potter series, Hogwarts School of Witchcraft and Wizardry
is where Harry Potter and his friends attend school. The school is located in a magical forest near
London and is known for its excellent education in magic and wizardry. Students at Hogwarts are
required to study a variety of subjects, including potion-making, herbology, history of magic, and
defense against the dark arts. Additionally, students must participate in physical education classes
and learn how to dance the troll. Hogwarts is also home to many famous professors, such as Albus
Dumbledore and Severus Snape, who teach and mentor the students. Overall, attending Hogwarts is a
unique and exciting experience that allows students to develop their skills and knowledge in the art
of magic.
CPU times: user 32.8 s, sys: 0 ns, total: 32.8 s
Wall time: 32.7 s
